Testing Pinecone:

In [ ]:
# %pip install pinecone
# %pip install python-dotenv
# %pip install mysql-connector-python
# %pip install -U langchain-google-genai
# %pip install PyMySQL

In [122]:
import warnings
warnings.filterwarnings("ignore")

In [162]:
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables

True

In [4]:
from pinecone import Pinecone, ServerlessSpec

In [126]:

PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")
# "pcsk_5YddPH_Aup51d2Gda1sWKwohEc1ZKHeEUC6yC4rHzNts9RPWZcQ8Tw6aqZsnMyGZMQCkqM"

pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
## My Sql Connection:


In [ ]:
import mysql.connector
import pandas as pd


True

In [164]:
# loading the mysql database credentials
host = os.getenv("host")
user = os.getenv("user")
password = os.getenv("password")
database = os.getenv("database")
port = os.getenv("port")


In [136]:

# Step 1: Connect to your local MySQL database
conn = mysql.connector.connect(
    host=host,      
    user=user,   
    password=password,  
    database=database   
)

# Step 2: Create a cursor
cursor = conn.cursor()


In [137]:

# Step 3: Write your SQL query
query = "SELECT * FROM workexperience;"  # example table

# Step 4: Use pandas to read the result into a DataFrame
df = pd.read_sql(query, conn)

# Step 5: Display the DataFrame
# print(df)

# Optional: Close the connection when done
cursor.close()
conn.close()


In [138]:
df.head(2)

,workExperienceId,company,role,startDate,endDate,description,locationCity,locationCountry
0,EXP001,Tata Consultancy Services,Software Engineer,2021-01-10,2023-05-20,Worked on backend systems and REST APIs using ...,Mumbai,India
1,EXP002,Google,Data Analyst,2019-06-01,2021-12-15,Analyzed user data to generate actionable insi...,Hyderabad,India


## Testing llm 

In [23]:
# %pip install -q -U google-genai

In [155]:
from google import genai
google_api_key= os.getenv("llm_api_key")

client = genai.Client(api_key=google_api_key)

In [158]:
response = client.models.generate_content(
    # model="gemini-2.0-flash",
    model = "gemini-2.5-flash",
    contents="Hi, good morning!!",
)

print(response.text)


Good morning to you too! How can I help you today?


In [35]:
from sqlalchemy import create_engine
# Connecting to the my-sql workbench:
from langchain.utilities import SQLDatabase
import pymysql


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    google_api_key = google_api_key,
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [165]:
from langchain.utilities import SQLDatabase
import pymysql
from urllib.parse import quote_plus
encoded_password = quote_plus(password)
engine_url = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}:{port}/{database}"
db = SQLDatabase.from_uri(engine_url)

In [166]:
# check that the database has been instantiated correctly
db.get_usable_table_names()

['education',
 'mystartup_users',
 'mystartupuserskills',
 'personalinformation',
 'skills',
 'userresume',
 'workexperience']

In [167]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [168]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [101]:
output = agent_executor.run(
    "How many candidates are there in the education table"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:education, mystartup_users, mystartupuserskills, personalinformation, skills, userresume, workexperienceI should query the schema of the education table to see what columns are available.
Action: sql_db_schema
Action Input: education
CREATE TABLE education (
	`educationId` VARCHAR(50) NOT NULL, 
	degree VARCHAR(100), 
	major VARCHAR(100), 
	school VARCHAR(100), 
	`startDate` DATE, 
	`endDate` DATE, 
	grade VARCHAR(20), 
	PRIMARY KEY (`educationId`)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from education table:
educationId	degree	major	school	startDate	endDate	grade
EDU001	Bachelor of Science	Computer Science	IIT Delhi	2017-07-01	2021-06-30	8.5
EDU002	Bachelor of Technology	Electrical Engineering	NIT Trichy	2016-08-01	2020-05-15	8.2
EDU003	Master of Science	Data Science	BITS Pilani	2021-09-01	2023-07-01	9.0
*/I should count the number of rows in the education table.
A

In [106]:
output

'There are 5 candidates in the education table.'